## Import Libraries

In [1]:
# %matplotlib ipympl
# %matplotlib inline
%matplotlib wx

In [2]:
import matplotlib.pyplot as plt
plt.ion()

In [3]:
from pydgilib_extra import *
from atprogram.atprogram import atprogram

In [4]:
from os import getcwd, path, pardir

## Compile and program project

In [5]:
project_path = path.join(getcwd(), "AES-S")
project_path

'C:\\Users\\erikw_000\\Documents\\GitHub\\Atmel-SAML11\\Experiments\\AES\\AES-S'

In [6]:
atprogram(project_path, verbose=2)

make: Nothing to be done for 'all'.

[DEBUG] Starting execution of "chiperase"
[DEBUG] Starting process 'C:\Program Files (x86)\Atmel\Studio\7.0\atbackend\atbackend.exe'
[DEBUG] Connecting to TCP:127.0.0.1:3498
[INFO] Connected to edbg, fw version: 3.25
[INFO] Firmware check OK
[DEBUG] Command "chiperase" finished with return code 0
[DEBUG] Starting execution of "program"
[DEBUG] Memory segment base written at 0x00000000. Size = 0x00004310.
[DEBUG] Memory segment base written at 0x00004310. Size = 0x00000068.
[DEBUG] Memory segment base written at 0x0000fb00. Size = 0x00000020.
[DEBUG] Memory segment base written at 0x00804000. Size = 0x00000020.
[DEBUG] Command "program" finished with return code 0
[DEBUG] Exit successfully.
Firmware check OK
Chiperase completed successfully
Programming completed successfully.



0

## Data Logging

Create a figure for the plot.

In [7]:
fig = plt.figure(figsize=(10, 6))
fig.show()

Create the configuration dictionary for `DGILibExtra`.

In [8]:
config_dict = {
    "interfaces": [INTERFACE_POWER, INTERFACE_GPIO],
    "power_buffers": [{"channel": CHANNEL_A, "power_type": POWER_CURRENT}],
    "read_mode": [True, True, True, True],
    "loggers": [LOGGER_OBJECT, LOGGER_PLOT, LOGGER_CSV],
    "plot_pins": [False, False, True, True],
    "plot_pins_values": [True, True, True, True],
    "gpio_delay_time" : 0.0007,
    "plot_pins_method": "line",
    "plot_xmax": 15,
    "fig": fig,
    "window_title": "Experiment AES",
    "file_name_base": "experiment_aes"
}

Stop criteria to pass to the logger:

In [9]:
#MIN_LOG_DURATION = 0.5 # seconds
def stop_fn(logger_data):
    return all(logger_data.gpio.values[-1]) #and logger_data.gpio.timestamps[-1] > MIN_LOG_DURATION
# def stop_fn(logger_data):
#     return len(logger_data.gpio) > 30

Perform the measurement.

In [10]:
data = []
fig.clf()
for ax in fig.get_axes():
    ax.cla()

with DGILibExtra(**config_dict) as dgilib:
    dgilib.device_reset()
#     data = dgilib.logger.log(config_dict["plot_xmax"],stop_fn)
    dgilib.logger.log(100,stop_fn)
    data = dgilib.data

## Analysis

Create Stop Function to stop parsing the data when all pins are high.

In [ ]:
def stop_function(pin_values):
    return all(pin_values)

Parse the data.

In [11]:
encrypt_charge, encrypt_time = power_and_time_per_pulse(data, 2, stop_function=stop_function)

In [12]:
decrypt_charge, decrypt_time = power_and_time_per_pulse(data, 3, stop_function=stop_function)

In [13]:
MBEDTLS_AES_BLOCK_SIZE = 16
MIN_AES_BLOCKS = 1
num_bytes = range(MIN_AES_BLOCKS * MBEDTLS_AES_BLOCK_SIZE, MBEDTLS_AES_BLOCK_SIZE * (MIN_AES_BLOCKS + len(encrypt_charge)), MBEDTLS_AES_BLOCK_SIZE)
print(f"MAX_AES_BLOCKS: {MIN_AES_BLOCKS + len(encrypt_charge) - 1}")

MAX_AES_BLOCKS: 100


In [14]:
from lmfit import Model

def line(x, slope, intercept):
    """a line"""
    return [slope*i + intercept for i in x]

mod = Model(line)
pars = mod.make_params(slope=0, intercept=1)

In [15]:
results = []
for y in [encrypt_charge, decrypt_charge, encrypt_time, decrypt_time]:
    result = mod.fit(y, pars, x=num_bytes)
    print(result.fit_report())
    results.append(result)

[[Model]]
    Model(line)
[[Fit Statistics]]
    # fitting method   = leastsq
    # function evals   = 7
    # data points      = 100
    # variables        = 2
    chi-square         = 4.5300e-14
    reduced chi-square = 4.6224e-16
    Akaike info crit   = -3529.06501
    Bayesian info crit = -3523.85467
[[Variables]]
    slope:      3.2370e-08 +/- 4.6550e-12 (0.01%) (init = 0)
    intercept:  7.2079e-09 +/- 4.3324e-09 (60.11%) (init = 1)
[[Correlations]] (unreported correlations are < 0.100)
    C(slope, intercept) = -0.868

[[Model]]
    Model(line)
[[Fit Statistics]]
    # fitting method   = leastsq
    # function evals   = 7
    # data points      = 100
    # variables        = 2
    chi-square         = 6.5289e-14
    reduced chi-square = 6.6622e-16
    Akaike info crit   = -3492.51158
    Bayesian info crit = -3487.30124
[[Variables]]
    slope:      3.3358e-08 +/- 5.5883e-12 (0.02%) (init = 0)
    intercept:  2.7969e-09 +/- 5.2011e-09 (185.96%) (init = 1)
[[Correlations]] (unre

In [16]:
fig2 = plt.figure(figsize=(9, 6))
fig2.canvas.set_window_title("Analysis AES")

In [17]:
charge_color = 'r'
time_color = 'b'

In [18]:
fig2.clf()
# fig2.suptitle("Energy analysis of AES")
ax1 = fig2.add_subplot(1, 1, 1)
ax2 = ax1.twinx()
ax1.set_xlabel('Number of bytes')
ax1.set_ylabel('Total Charge [C]', color=charge_color)
ax2.set_ylabel('Time [s]', color=time_color)
ax1.tick_params('y', colors=charge_color)
ax2.tick_params('y', colors=time_color)

In [19]:
lines = []
lines += ax1.plot(num_bytes, encrypt_charge, charge_color+'-', label='Encrypt Charge')
lines += ax1.plot(num_bytes, decrypt_charge, charge_color+'--', label='Decrypt Charge')
lines += ax2.plot(num_bytes, encrypt_time, time_color+'-', label='Encrypt Time')
lines += ax2.plot(num_bytes, decrypt_time, time_color+'--', label='Decrypt Time')
ax1.legend(handles=lines)
ax1.set_title(f"Encrypt Charge: Base {results[0].params['intercept'].value:.03} C plus {results[0].params['slope'].value:.03} mC per byte\n" +
             f"Decrypt Charge: Base {results[1].params['intercept'].value:.03} C plus {results[1].params['slope'].value:.03} mC per byte\n" +
             f"Encrypt Time: Base {results[2].params['intercept'].value:.03} s plus {results[2].params['slope'].value:.03} s per byte\n" +
             f"Decrypt Time: Base {results[3].params['intercept'].value:.03} s plus {results[3].params['slope'].value:.03} s per byte\n")
fig2.tight_layout()
fig2.show()

In [21]:
# Save Charge amount list into pickle file
import pickle
pickle.dump(encrypt_charge, open("aes_encrypt_charge.p", "wb"))
pickle.dump(decrypt_charge, open("aes_decrypt_charge.p", "wb"))

# Python script to do measurement

In [39]:
%%writefile experiment_aes.py

from os import getcwd, path
import pickle

from pydgilib_extra import *
from atprogram.atprogram import atprogram

def experiment_aes(project_root=getcwd(), verbose=1):
    project_path = path.join(project_root, "AES-S")
    atprogram(project_path, verbose=verbose)

    config_dict = {
        "interfaces": [INTERFACE_POWER, INTERFACE_GPIO],
        "power_buffers": [{"channel": CHANNEL_A, "power_type": POWER_CURRENT}],
        "read_mode": [True, True, True, True],
        "loggers": [LOGGER_OBJECT, LOGGER_CSV],
        "gpio_delay_time" : 0.0007,
        "file_name_base": "experiment_aes",
        "log_folder": project_root,
        "verbose": verbose-1
    }
    
    def stop_fn(logger_data):
        return all(logger_data.gpio.values[-1])
    
    if verbose:
        print(f"Start DGILibExtra with config: \n{config_dict}\n")

    data = []
    with DGILibExtra(**config_dict) as dgilib:
        dgilib.device_reset()
        dgilib.logger.log(1000,stop_fn)
        data = dgilib.data
    
    if verbose:
        print(f"DGILibExtra data: {data}")

    def stop_function(pin_values):
        return all(pin_values)

    encrypt_charge, encrypt_time = power_and_time_per_pulse(data, 2, stop_function=stop_function)
    decrypt_charge, decrypt_time = power_and_time_per_pulse(data, 3, stop_function=stop_function)

    # Save Charge amount list into pickle file
    pickle.dump(encrypt_charge, open(path.join(project_root, "aes_encrypt_charge.p"), "wb"))
    pickle.dump(decrypt_charge, open(path.join(project_root, "aes_decrypt_charge.p"), "wb"))
    
    if verbose:
        print(f"Saved results in: {path.join(project_root)}")

if __name__ == "__main__":
   experiment_aes()

Overwriting experiment_aes.py
